<a href="https://colab.research.google.com/github/hasan-sayeed/hellow-world/blob/master/ML_topo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sklearn

In [ ]:
tensorflow_version 2.x  # this line is not required unless you are in a notebook

In [10]:
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

#for Model
import time
import sys
tooSmall=10

#for ModelRun
from tensorflow.python.client import timeline
import argparse
import sys
import time
from sklearn import preprocessing
parser = argparse.ArgumentParser()
FLAGS = None
TYPE=np.float32

**myInputFn.py**

In [11]:
class Scaler:
    def __init__(self, d):
        self.scaler={}
        for key in d.keys():
            self.scaler[key] = preprocessing.StandardScaler()
            self.scaler[key].fit(d[key].reshape(-1,1))
    def __call__(self, x):
        X={}
        for key in x.keys():
            X[key]=self.scaler[key].transform(x[key].reshape(-1,1),copy=True).reshape(-1)
        return X
    def rescale(self, x): 
        X={}
        for key in x.keys():
            X[key]=self.scaler[key].inverse_transform(x[key].reshape(-1,1),copy=True).reshape(-1)
        return X
    def rescaleAs(self, key, x):
        return self.scaler[key].inverse_transform(x.reshape(-1,1),copy=True).reshape(-1)

class MyInputFn:
    def __init__(self, scaler, x, y=None):
        self.scaler=scaler
        self.x={}
        for key in x.keys():
            self.x[key]=np.copy(x[key])
        if y is not None:            
            self.y={}
            for key in y.keys():
                self.y[key]=np.copy(y[key])
        else:
            self.y=y
        
    def __call__(self, 
                 num_epochs=1,
                 shuffle=False,
                 batch_size=128,
                 queue_capacity=1000,
                 num_threads=1):
        x=self.scaler(self.x)
        if self.y is None:
            y=None
        else:
            y=self.scaler(self.y)
            y=y[y.keys().pop()] # from single y dict to array
        return tf.estimator.inputs.numpy_input_fn(x, y, batch_size, num_epochs,shuffle,queue_capacity,num_threads)
    
    def target(self):        
        y=self.y
        y=y[y.keys().pop()] # from y dict to array        
        return y

**model**

In [8]:
def createModelFromInput_fn(train_input_fn, hiddenLayers, maxSteps):
    featureColumns=tf.contrib.learn.infer_real_valued_columns_from_input_fn(train_input_fn)
    optimizer=tf.train.AdamOptimizer()
    regressor = tf.contrib.learn.DNNRegressor(feature_columns=featureColumns, hidden_units=hiddenLayers,optimizer=optimizer,activation_fn=tf.nn.tanh)
    regressor.fit(input_fn=train_input_fn, max_steps=maxSteps) 
    return regressor

def Score(regressor, input_fn):
    # https://www.tensorflow.org/api_docs/python/tf/contrib/learn/MetricSpec
    return regressor.evaluate(input_fn=input_fn, steps=1, 
                           metrics={
                               'MSE': tf.contrib.metrics.streaming_mean_squared_error
                           })

def SelfConsistantCycle(directRegressor, inverseRegressor, scaler, omegaSweep_input_fn_Inverse):    
    from myInputFn import MyInputFn
    from myInputFn import Scaler
    omegaSweep=np.copy(omegaSweep_input_fn_Inverse.x['omega'])
    gammaSweep=np.copy(omegaSweep_input_fn_Inverse.x['gamma'])
    trendSweep=np.copy(omegaSweep_input_fn_Inverse.x['trend'])
    # https://github.com/tensorflow/tensorflow/issues/9505
    chiPredicted=scaler.rescaleAs('chi',np.asarray(list(inverseRegressor.predict_scores(input_fn=omegaSweep_input_fn_Inverse(num_epochs=1,shuffle=False))))) # predicted is normalized now rescale
    predict_input_fn_Direct = MyInputFn( scaler,
        {'chi': chiPredicted, 'gamma': gammaSweep, 'trend': trendSweep})        
    omegaPredicted=scaler.rescaleAs('omega',np.asarray(list(directRegressor.predict_scores(input_fn=predict_input_fn_Direct(num_epochs=1,shuffle=False))))) # predicted is normalized now rescale
    return chiPredicted,omegaPredicted

**Model**

In [12]:
class Model:
    
    def __init__(self, dataset, hiddenLayers, maxSteps,model_dir, optimizer, debug):
        self.ok=False
        if len(dataset['chi'])<tooSmall: # too few points for ml watch out as there will be 10 point just for gamma
            return        
        self.debug=debug
        from myInputFn import Scaler
        # scaler
        self.scaler=Scaler(dataset) 
        # split
        from sklearn import model_selection
        featKeys=[]
        featTest=[]
        featTrain=[]
        if debug:
            rnd=4852
        else:
            rnd=np.random.randint((2**31)*2)
        
        for key, val in dataset.iteritems():
            train,test= model_selection.train_test_split(val, test_size=0.2 , random_state=rnd)
            if key=='chi' or key=='omega':
                if key=='omega':
                    omegaTrain=train
                    omegaTest=test
                else:
                    chiTrain=train
                    chiTest=test
            else:                
                featKeys.append(key)
                featTest.append(test)
                featTrain.append(train)
    
        from myInputFn import MyInputFn
        
        
        # Direct regressor
        directTrainFeatures={'chi': chiTrain}
        for i,key in enumerate(featKeys):
            directTrainFeatures[key]=featTrain[i]
            
        train_input_fn_Direct = MyInputFn( self.scaler,
            directTrainFeatures,
            {'omega':omegaTrain})
        
        startTime = time.time()    
        self.directRegressor = self.createModelFromInput_fn(train_input_fn_Direct(num_epochs=None,shuffle=True), hiddenLayers, maxSteps,model_dir+"/direct", optimizer)
        print('# Direct Fitting time: %f' % (time.time() - startTime))

        # test direct regressor
        directTestFeatures={'chi': chiTest}
        for i,key in enumerate(featKeys):
            directTestFeatures[key]=featTest[i]
        
        test_input_fn_Direct = MyInputFn( self.scaler,
            directTestFeatures,
            {'omega':omegaTest})

        from model import Score        
        self.scoreDirect=self.Score(self.directRegressor,test_input_fn_Direct(num_epochs=1,shuffle=False))
        print'# Direct MSE: '+str(self.scoreDirect['MSE'])

        # Inverse regressor
        invTrainFeatures={'omega': omegaTrain}
        for i,key in enumerate(featKeys):
            invTrainFeatures[key]=featTrain[i]
        
        train_input_fn_Inverse = MyInputFn( self.scaler,
            invTrainFeatures,
            {'chi':chiTrain})
        startTime = time.time()    
        self.inverseRegressor = self.createModelFromInput_fn(train_input_fn_Inverse(num_epochs=None,shuffle=True), hiddenLayers, maxSteps,model_dir+"/inverse", optimizer)
        print('# Inverse Fitting time: %f' % (time.time() - startTime))

        invTestFeatures={'omega': omegaTest}
        for i,key in enumerate(featKeys):
            invTestFeatures[key]=featTest[i]
        
        test_input_fn_Inverse = MyInputFn( self.scaler,
            invTestFeatures,
            {'chi':chiTest})        
        self.scoreInverse=self.Score(self.inverseRegressor,test_input_fn_Inverse(num_epochs=1,shuffle=False))
        print'# Inverse MSE: '+str(self.scoreInverse['MSE'])    
        
        # calc omega range BUG BUG 1 d lin
        if dataset.has_key('gamma'):
            self.OmegaMinGamma(dataset['omega'], dataset['gamma'])
        self.isChiPositive=np.min(dataset['chi'])>0.0
        self.ok=True

    def createModelFromInput_fn(self, train_input_fn, hiddenLayers, maxSteps, model_dir, optimizer):
        if self.debug:
            rnd=87897978
            cores=1
        else:
            rnd=None
            cores=0            
        my_checkpointing_config = tf.estimator.RunConfig(
            save_checkpoints_steps=maxSteps/10,
            keep_checkpoint_max = 10,       # Retain the 10 most recent checkpoints.
            tf_random_seed=rnd,
            # num_cores= cores
        )
                
        if(optimizer=='grad'):
            opt=tf.train.GradientDescentOptimizer(0.001)
        elif(optimizer=='prox'):
            opt=tf.train.ProximalGradientDescentOptimizer(0.001)
        else:
            opt=tf.train.AdamOptimizer()
        featureColumns=tf.contrib.learn.infer_real_valued_columns_from_input_fn(train_input_fn)
        regressor = tf.contrib.learn.DNNRegressor(feature_columns=featureColumns, hidden_units=hiddenLayers,optimizer=opt,activation_fn=tf.nn.tanh, model_dir=model_dir,config=my_checkpointing_config)
        regressor.fit(input_fn=train_input_fn, max_steps=maxSteps) 
        return regressor
    
    def Score(self, regressor, input_fn):
        # https://www.tensorflow.org/api_docs/python/tf/contrib/learn/MetricSpec
        return regressor.evaluate(input_fn=input_fn, steps=1, 
                               metrics={
                                   'MSE': tf.contrib.metrics.streaming_mean_squared_error
                               })
    def OmegaMinGamma(self, omega, gamma):
        u,v=np.unique(gamma, True)
        gammaVal=[]
        omegaMin=[]
        omegaMax=[]
        s=None
        for x in v: # select each gamma group
            if s is not None: # find offsets of gamma groups
                gammaVal.append(gamma[s])
                omegaMin.append(np.min(omega[s:x]))
                omegaMax.append(np.max(omega[s:x]))
            s=x
        gammaVal.append(gamma[s])
        omegaMin.append(np.min(omega[s:]))
        omegaMax.append(np.max(omega[s:]))
        # regression
        if len(gammaVal)<2:
            self.slopeOmn, self.interceptOmn = 0.0,np.min(omega)
            self.slopeOmx, self.interceptOmx=0.0,np.max(omega)
        else:
            from scipy import stats
            self.slopeOmn, self.interceptOmn, r_value, p_value, std_err =stats.linregress(gammaVal,omegaMin)
            self.slopeOmx, self.interceptOmx, r_value, p_value, std_err =stats.linregress(gammaVal,omegaMax)
        return {'gamma':np.array(gammaVal).reshape(-1), 'omegaMin':np.array(omegaMin).reshape(-1), 'omegaMax':np.array(omegaMax).reshape(-1)}
    
    def omegaRange(self, gamma):
        omn=self.slopeOmn*gamma+ self.interceptOmn
        omx=self.slopeOmx*gamma+ self.interceptOmx
        return omn, omx
    
    def SelfConsistantCycle(self, omegaSweep_input_fn_Inverse):    
        from myInputFn import MyInputFn
        from myInputFn import Scaler
        import math
        chiPredicted=self.scaler.rescaleAs('chi',np.asarray(list(self.inverseRegressor.predict_scores(input_fn=omegaSweep_input_fn_Inverse(num_epochs=1,shuffle=False))))) # predicted is normalized now rescale
        chiUpperBoundIdx=np.abs(chiPredicted)<math.pi # keep only valid chi and correct 
        if self.isChiPositive:
            chiLowerBoundIdx=chiPredicted>0.0 
        else:
            chiLowerBoundIdx=chiPredicted<0.0 
        validChiIdx=np.logical_and(chiLowerBoundIdx,chiUpperBoundIdx)   
        chiPredicted=chiPredicted[validChiIdx]
        if(np.count_nonzero(chiPredicted)<2): # too few points
            return None,None, None 
        features={}

        for key in omegaSweep_input_fn_Inverse.x.keys():
            if key != 'omega':
                features[key]=np.copy(omegaSweep_input_fn_Inverse.x[key])[validChiIdx]
            else:
                deviation=np.copy(omegaSweep_input_fn_Inverse.x[key])[validChiIdx] # original omegas
        # should it return here?        
        features['chi']=chiPredicted
        predict_input_fn_Direct = MyInputFn( self.scaler, features)        
        omegaPredicted=self.scaler.rescaleAs('omega',np.asarray(list(self.directRegressor.predict_scores(input_fn=predict_input_fn_Direct(num_epochs=1,shuffle=False))))) # predicted is normalized now rescale
        deviation=omegaPredicted[:]-deviation[:]
        return chiPredicted,omegaPredicted, deviation

def padding(x):
        mid=len(x)/2
        X=np.append(-x[mid:0:-1]+2*x[0],x) 
        X=np.append(X,-x[-2:mid:-1]+2*x[-1])
        return X

def extendSymetrically(dataset,targetKey):
    for i,key in enumerate(dataset):
        if(key!=targetKey):
            mid=len(dataset[key])/2
            # rightmost half
            # y0=dataset[targetKey][-1]
            extend=dataset[key][:mid-1:-1]            
            x0=extend[0]
            extend=x0-(extend-x0)
            extendY=dataset[target][:mid-1:-1]
            y0=extendY[0]
            extendY=y0-(extendY-y0)
            # BUG BUG multi D ??
            
    return
def test():
    def omega(x):
        return 1.0+x*x+x*x*x
    def dOmega(x):
        return x*(2+3*x)
    def trend(x):
        return np.sign(dOmega(x))
    def es():
        chi=np.linspace(-0.5,1.5,20, dtype=np.float32)
        ds={'chi':chi, 'omega':omega(chi)}
        extendSymetrically(ds,'omega')
        
    es()
    import matplotlib.pyplot as plt
    hiddenLayers=[122,122,122,122]
    maxSteps=1500
    model_dir='/tmp/test-model/'
    chi=np.linspace(-0.5,1.5,20, dtype=np.float32)    
    om=omega(chi)

    plt.show()
    datasetS={'chi':chi,'omega':om}
    modelS=Model(datasetS, hiddenLayers, maxSteps,model_dir+'S/')
    # extend by reflection
    X0=chi[-1]
    ihc=chi[::-1]-X0
    ihc=-ihc+X0
    ihc=np.append(chi,ihc)
    
    Y0=om[-1]
    mo=om[::-1]-Y0
    mo=-mo+Y0
    mo=np.append(om,mo)
    
    datasetR={'chi':ihc,'omega':mo}    
    modelR=Model(datasetR, hiddenLayers, maxSteps,model_dir+'R/')
    from myInputFn import MyInputFn
    input_fn_S = MyInputFn(modelS.scaler,
            {'chi': np.linspace(-0.5,1.5,200, dtype=np.float32)})                                
    predictS=modelS.scaler.rescaleAs('omega',np.asarray(list(modelS.directRegressor.predict_scores(input_fn=input_fn_S(num_epochs=1,shuffle=False))))) # predicted is normalized now rescale
    input_fn_R = MyInputFn(modelR.scaler,
                           {'chi': np.linspace(-0.5,1.5,200, dtype=np.float32)})                                    
    predictR=modelR.scaler.rescaleAs('omega',np.asarray(list(modelR.directRegressor.predict_scores(input_fn=input_fn_R(num_epochs=1,shuffle=False))))) # predicted is normalized now rescale

    plt.plot(input_fn_R.x['chi'],predictR, 'r+')
    plt.plot(input_fn_S.x['chi'],predictS, 'b+')
    plt.plot(chi, om, 'bo')
    plt.plot(ihc, mo, 'g-')
    # plt.plot(ihc, mo, 'g+')
    plt.show()
    
# test()

SyntaxError: ignored

**ModelRun**

In [15]:
def main(_):
    from Model import padding
    # mode from 1 ... ncol-1 , trend next col NEW last col gamma
    def readData(fileName):
        def mkarray(x):
            return np.array([x],dtype=TYPE)
        infile=open(fileName) if len(fileName) else sys.stdin 
        dataRaw=np.array([ np.array([mkarray(token) for token in line.split()]) for line in infile.readlines()])
        chi=dataRaw[:, 0, None].reshape(-1) 
        indexes=chi>0.0
        datasetPositiveChi=dataRaw[indexes,:]
        indexes=chi<0.0
        datasetNegativeChi=dataRaw[indexes,:]
        
        return datasetPositiveChi,datasetNegativeChi
    class SelectMode:
        def __init__(self,padding):
            self.padding=padding
        def __call__(self,data, mode, trendSelected):    
            ds=self._selectMode(data, mode, trendSelected)
            if not self.padding:
                return ds
            
            gamma=ds['gamma']
            u,idx=np.unique(gamma, True) # values and indexes returned
            s=None
            chiOriginal=ds['chi']
            omegaOriginal=ds['omega']
            chiPadded=[]
            omegaPadded=[]
            gammaPadded=[] # this is not really padded just corrected for size
            for x in idx:
                if s is not None:
                    chi=chiOriginal[s:x]
                    chi=padding(chi)
                    gm=np.zeros_like(chi)
                    gm[:]=gamma[s] # enlarge gamma and fill with correct value
                    gammaPadded.append(gm)
                    chiPadded.append(chi)
                    omega=omegaOriginal[s:x]
                    omegaPadded.append(padding(omega))
                s=x
            # repeat one last time
            chi=chiOriginal[s:]
            chi=padding(chi)
            gm=np.zeros_like(chi)
            gm[:]=gamma[s] 
            gammaPadded.append(gm)
            chiPadded.append(chi)
            omega=omegaOriginal[s:]
            omegaPadded.append(padding(omega))
            gammaPadded=np.array(gammaPadded)
            gammaPadded=gammaPadded.reshape(-1)            
            chiPadded=np.array(chiPadded)
            chiPadded=chiPadded.reshape(-1)
            omegaPadded=np.array(omegaPadded)
            omegaPadded=omegaPadded.reshape(-1)
            fp=open('padded.chi.dat','ab')
            np.savetxt(fp,chiPadded)
            fp.close()
            fp=open('padded.om.dat','ab')
            np.savetxt(fp,omegaPadded)
            fp.close()
            print 'saved'
            chiPadded=np.array(chiPadded).reshape(-1)
            omegaPadded=np.array(omegaPadded).reshape(-1)
            gammaPadded=np.array(gammaPadded).reshape(-1)
            return {'chi':chiPadded,'omega':omegaPadded,'gamma':gammaPadded}
        
        
        def _selectMode(self,data, mode, trendSelected):    
            modeCol=1+(mode-1)*2
            # indexes=[len(rec)>(modeCol+1) for rec in dataRaw]
            # data=dataRaw[indexes]
            chi=data[:, 0, None] 
            omega=data[:, modeCol ,None] 
            gamma=data[:, -1, None] # last column is gamma
            trend=data[:, (modeCol+1) ,None] # slope field follows omega field
            trend=np.sign(trend)
            indexes=trend==trendSelected
            return {'chi':chi[indexes],'omega':omega[indexes], 'gamma':gamma[indexes]}        
        
    # prepare from flags
    hiddenLayers= eval('['+FLAGS.hiddenLayers+']')
    model_dir=FLAGS.export+'/'+FLAGS.hiddenLayers.replace(',','-')+'/'
    maxSteps = eval('int('+FLAGS.maxSteps+')')
    omegaStep=eval('float('+FLAGS.omegaStep+')')
    outfile=sys.stdout if FLAGS.output=='' else open(FLAGS.output,'w')
    outfile.write('# data: '+ FLAGS.data +"\n")
    outfile.write('# hiddenLayers: ['+ FLAGS.hiddenLayers+']' +"\n")
    outfile.write('# maxSteps: '+ FLAGS.maxSteps +"\n")
    outfile.write('# optimizer: '+ FLAGS.opt +"\n")
    run_metadata=tf.RunMetadata()
    def comment(): # not working
        trace = timeline.Timeline(step_stats=run_metadata.step_stats)
        # chrome://tracing
        # https://github.com/tensorflow/tensorflow/issues/1824#issuecomment-225754659
        # https://www.tensorflow.org/performance/performance_guide
        import os
        try:
            os.stat(model_dir)
        except:
            os.mkdir(model_dir)   
        trace_file = open(model_dir+'timeline.ctf.json', 'w')
    debug= True if FLAGS.debug else False
    if debug:
        os.environ['CUDA_VISIBLE_DEVICES'] = ''
        
    optimizer=FLAGS.opt
    if(FLAGS.opt=='grad'):
        model_dir=model_dir+'grad/'
    elif(FLAGS.opt=='prox'):
        model_dir=model_dir+'prox/'
    else:
        model_dir=model_dir # +'adam/'
        
    datasetPositiveChi,datasetNegativeChi=readData(FLAGS.data)
    padded=True if FLAGS.pad else False
    selectMode=SelectMode(padded)
    from Model import Model
    model={}
    modeRange= eval('range('+FLAGS.modeRange+')' ) if ',' in FLAGS.modeRange else [int(FLAGS.modeRange)]  # else single mode value    
    # create seperate models for each mode & trend with extra gamma feature 
    
    for mode in modeRange:
        for trend in range(-1,2,2):
            nm='p_'+str(mode)+'_'+str(trend)
            m=Model(selectMode(datasetPositiveChi, mode, trend), hiddenLayers, maxSteps, model_dir+nm, optimizer, debug) # positive only
            # trace_file.write(trace.generate_chrome_trace_format())
            if m.ok :
                model[nm]=m
            nm='n_'+str(mode)+'_'+str(trend)
            m=Model(selectMode(datasetNegativeChi, mode, trend), hiddenLayers, maxSteps, model_dir+nm, optimizer, debug)
            
            if m.ok :
                model[nm]=m
    # run models one for each mode & trend with extra gamma feature     
    
    from myInputFn import MyInputFn
    gammaValues= eval('np.linspace('+FLAGS.gammaRange+', dtype=TYPE)' ) if ',' in FLAGS.gammaRange else [TYPE(FLAGS.gammaRange)]  # else single gamma value    
    omegaTarget= eval('np.linspace('+FLAGS.omegaRange+', dtype=TYPE)' ) if ',' in FLAGS.omegaRange else None  # else auto 
    if omegaTarget is not None:
        outfile.write('# omegaRange: '+ FLAGS.omegaRange +"\n")
    trendFilter =False if FLAGS.trendFilter_off else True
    if trendFilter:
        outfile.write('# trendFilter ON' +"\n")
    
    for mode in modeRange:
        for trend in range(-1,2,2):
            for s in ['p','n']:
                nm=s+'_'+str(mode)+'_'+str(trend)
                if nm in model:
                    for gammaValue in gammaValues:
                        if omegaTarget is None:
                            omegaMin,omegaMax=model[nm].omegaRange(gammaValue)  
                            omegaPts=abs(int((omegaMin-omegaMax)/omegaStep))
                            omegaSweep=np.linspace(omegaMin,omegaMax, omegaPts)
                        else:
                            omegaSweep=np.copy(omegaTarget)
                        # LP cleanup - do a small step in omega to test trend
                        if trendFilter:
                            deltaOmega=np.abs(omegaSweep[1]-omegaSweep[0])*1e-3
                            tmp=np.append(omegaSweep, omegaSweep)
                            tmp[::2]=omegaSweep[:]
                            tmp[1::2]=omegaSweep[:]+deltaOmega
                            omegaSweep=tmp
                        print "# model: "+nm+" gammaValue: "+str(gammaValue)
                        gammaSweep=np.zeros_like(omegaSweep)
                        gammaSweep[:]=gammaValue
                        omegaSweep_input_fn_Inverse = MyInputFn(model[nm].scaler, # not great
                            {'omega': omegaSweep, 'gamma': gammaSweep})                                
                        chiPredicted,omegaPredicted,deviation=model[nm].SelfConsistantCycle(omegaSweep_input_fn_Inverse) # still need to do lp clean up 
                        if chiPredicted is not None:
                            if trendFilter:
                                trendPredicted=np.zeros_like(chiPredicted)
                                trendPredicted[:-1]=(omegaPredicted[1:]-omegaPredicted[:-1])/(chiPredicted[1:]-chiPredicted[:-1])
                                trendPredicted[-1]=trendPredicted[-2] # assume trend on last
                                trendPredicted=np.sign(trendPredicted)
                                goodTrendIdx= trendPredicted==trend
                                goodTrendIdx[1::2]=False # eliminate all deltaOmega
                            else:
                                goodTrendIdx=range(len(chiPredicted))
                            outfile.write("# "+nm+": omegaSC\tdeviation\tchi\tgamma\ttrend\tmode\n")
                            output(outfile,omegaPredicted[goodTrendIdx], deviation[goodTrendIdx], chiPredicted[goodTrendIdx], gammaValue, trend, mode)
                
def output(outfile, omegaSC, deviation,chi,gamma,trend, mode):

    for i in range(len(omegaSC)):
        outfile.write(str(omegaSC[i])+"\t"+str(deviation[i])+"\t"+str(chi[i])+"\t"+str(gamma)+"\t"+str(trend)+"\t"+str(mode)+"\n")
    outfile.write("\n\n")
if __name__ == '__main__':
    
    parser.add_argument('--maxSteps', type=str, default='25000',
                        help='maxSteps [25000] int ')   

    parser.add_argument('--omegaStep', type=str, default='0.00002',
                        help='omegaStep [0.00002] float ')   

    
    parser.add_argument('--hiddenLayers', type=str, default='131,131,131,131,131',
                        help='hiddenLayers [131,131,131,131,131] e.g. 12,12,12,12 as in list')   
        
    parser.add_argument('--data', type=str, default='',
                      help='Input file [stdin]')

    parser.add_argument('--export', type=str, default='/tmp/model',
                        help='Export model to path [/tmp/model]')
    
    parser.add_argument('--gammaRange', type=str, default='0.10,0.20,11',
                        help='gamma range [0.10,0.20,11] e.g. 10,12,300 as in np.linspace')   
    
    parser.add_argument('--modeRange', type=str, default='1,7',
                        help='mode index range [1,7] e.g. 1,2 as in range')   
    
    parser.add_argument('--omegaRange', type=str, default='',
                        help='omega range [auto] e.g. 5.67,5.7,100 as in linspace ')   
    
    parser.add_argument('--output', type=str, default='',
                        help='filename [stdout]')   
    
    parser.add_argument('--opt', type=str, default='',
                        help='optimizer [adam] e.g. grad | prox ')   

    parser.add_argument('--pad', action='store_true',
                        help='extend by padding')   
    
    parser.add_argument('--debug', action='store_true',
                        help='Fixed random seed and one CPU no GPU')   
    
    parser.add_argument('--trendFilter-off', action='store_true',
                        help='Trend filter off')   
    
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

SyntaxError: ignored